In [ ]:
# TEST RUN, VERY LOW BATCH AND DATA SIZES > SHOWS 2 SE

In [1]:
MLFLOW_URI = "https://mlflow.lewagon.ai/"
EXPERIMENT_NAME = "UK Lon lwb_smr vertex_run_02" # template
EXPERIMENT_TAGS = {
    'USER': 'hsth',
    'RUN NAME': 'test_vertex2, operational',
    'VERSION': 'M2_R04_15',
    'DESCRIPTION': 'test_Model VGG16 UNet, 20 epochs, 72k images',
    'LOSS': 'dice',
    'METRICS': 'accuracy, binaryIoU, AUC'
}

UNET_INPUT_SHAPE = (224,224,3)
BATCH_SIZE = 8
EPOCHS = 2
# LOSS='binary_crossentropy'
LOSS = 'DICE'
TEST_CHECKPOINT_PATH_N_FILE_NAME = ""
TEST_MODEL_PATH_N_FILE_NAME = ""

# TENSOR SLICE DATA LOADER

In [2]:
import os
import pandas as pd

In [3]:
!ls '../../../raw_data/'


checkpoints		models		       vertex_data_jpegs.zip
image_datasets_csv	train_RGB_tiles_jpeg
image_datasets_csv.zip	train_mask_tiles_jpeg


In [4]:
import os
import pandas as pd

x_path = '../../../raw_data/train_RGB_tiles_jpeg/'
x_images = os.listdir(x_path)
y_path = '../../../raw_data/train_mask_tiles_jpeg/'
y_masks = os.listdir(y_path)
        

In [5]:
root_path = '../../../raw_data/'
folders = ['train_RGB_tiles_jpeg', 'train_mask_tiles_jpeg']
folder_path = [f'{root_path}{folder}' for folder in folders]
folder_path

['../../../raw_data/train_RGB_tiles_jpeg',
 '../../../raw_data/train_mask_tiles_jpeg']

In [6]:
train_images, train_mask = [], []
# train_images =[f'../../../raw_data/train_RGB_tiles_jpeg/{filename}' for filename in os.listdir(folder_path[0])]
for i, filename in enumerate(os.listdir(folder_path[0])):
    if i == 2*BATCH_SIZE: break
    train_images.append(f'../../../raw_data/train_RGB_tiles_jpeg/{filename}')
    
# train_mask = [f'../../../raw_data/train_mask_tiles_jpeg/{filename}' for filename in os.listdir(folder_path[1])]
for i, filename in enumerate(os.listdir(folder_path[1])):
    if i == 2*BATCH_SIZE: break
    train_mask.append(f'../../../raw_data/train_mask_tiles_jpeg/{filename}')
    

In [7]:
train_images.sort()
train_mask.sort()


train_df = pd.DataFrame()
# train_df['file_path'] = train_images
train_df['image_path'] = train_images
train_df['mask_path'] = train_mask

len(train_df), train_df.head()

(16,
                                           image_path  \
 0  ../../../raw_data/train_RGB_tiles_jpeg/austin1...   
 1  ../../../raw_data/train_RGB_tiles_jpeg/austin2...   
 2  ../../../raw_data/train_RGB_tiles_jpeg/austin6...   
 3  ../../../raw_data/train_RGB_tiles_jpeg/austin7...   
 4  ../../../raw_data/train_RGB_tiles_jpeg/chicago...   
 
                                            mask_path  
 0  ../../../raw_data/train_mask_tiles_jpeg/austin...  
 1  ../../../raw_data/train_mask_tiles_jpeg/austin...  
 2  ../../../raw_data/train_mask_tiles_jpeg/chicag...  
 3  ../../../raw_data/train_mask_tiles_jpeg/chicag...  
 4  ../../../raw_data/train_mask_tiles_jpeg/kitsap...  )

In [8]:
# train_df.info()

In [9]:
# root_path = '/raw_data/'

# # raw_data/train_mask_tiles_jpeg

# x_path = root_path + 'train_RGB_tiles_jpeg/'
# x_images = os.listdir(x_path)
# y_path = root_path + 'train_mask_tiles_jpeg/'
# y_masks = os.listdir(y_path)
    

# folders = ['train_RGB_tiles_jpeg', 'train_mask_tiles_jpeg']
# folder_path = [f'{root_path}{folder}' for folder in folders]
# folder_path

# train_images, train_mask = [], []
# train_images = [f'{x_images}{filename}' for filename in os.listdir(folder_path[0])]
# train_mask = [f'{y_masks}{filename}' for filename in os.listdir(folder_path[1])]

# train_images.sort()
# train_mask.sort()


# train_df = pd.DataFrame()
# # train_df['file_path'] = train_images
# train_df['image_path'] = train_images
# train_df['mask_path'] = train_mask

# train_df.head()

In [10]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

def holdout(df, train_ratio=0.8, test_to_val_ratio=0.5, include_all=False):

    img_paths = df["image_path"].values
    msk_paths = df["mask_path"].values

    df_mask = df.copy()

    df_train, df_val = train_test_split(df_mask, train_size=train_ratio)
    df_test, df_val = train_test_split(df_val, test_size=test_to_val_ratio)

    ds_train = tf.data.Dataset.from_tensor_slices(
         (df_train["image_path"].values, df_train["mask_path"].values)
    )
    ds_val = tf.data.Dataset.from_tensor_slices(
        (df_val["image_path"].values, df_val["mask_path"].values)
    )
    ds_test = tf.data.Dataset.from_tensor_slices(
        (df_test["image_path"].values, df_test["mask_path"].values)
    )

    return ds_train, ds_val, ds_test

ds_train, ds_val, ds_test = holdout(train_df)

2022-06-12 04:54:57.454391: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-12 04:54:57.471360: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-12 04:54:57.471697: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-12 04:54:57.473218: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [11]:
def process_path(input_path, mask_path):
    """
    Load images from files.
    :input_path: the path to the satellite file
    :mask_path: the path to the mask file
    :return: The image and mask
    .. note:: Works with jpg images 
              Only the first channel is kept for the mask
    """
    
    IMAGE_SQ_SIZE = 224

    input_img = tf.io.read_file(input_path)   
    input_img = tf.io.decode_jpeg(input_img, channels=3)
    input_img =  tf.image.resize(input_img, [IMAGE_SQ_SIZE, IMAGE_SQ_SIZE])

    mask_img = tf.io.read_file(mask_path)   
    mask_img = tf.io.decode_jpeg(mask_img, channels=1)
    mask_img =  tf.image.resize(mask_img, [IMAGE_SQ_SIZE, IMAGE_SQ_SIZE])

   
    return input_img, mask_img

def normalize(image, mask):
    # image = tf.cast(image, tf.float32) / 255.

    return tf.math.divide(image, 255), tf.math.divide(mask, 255) 

In [12]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

ds_train = ds_train.map(process_path) \
.map(normalize) \
.batch(batch_size=BATCH_SIZE) \
.prefetch(buffer_size=AUTOTUNE)

ds_val = ds_val.map(process_path) \
.map(normalize) \
.batch(batch_size=BATCH_SIZE) \
.prefetch(buffer_size=AUTOTUNE)

### we do not use:
# CustomDataLoader

In [13]:
# from tensorflow import keras
# import tensorflow as tf
# import numpy as np
# import math
# import os
# from skimage.transform import resize
# from PIL import Image

# class CustomDataLoader(keras.utils.Sequence):
#     """ Allow custom data import and output from image files for masking """

#     def __init__(self, x_images, x_path, y_masks, y_path, input_image_size, batch_size):
#         """
#         x_images            is a list of RGB images in a directory
#         x_path              is the path for the images
#         y_masks             is a list of greyscale masks in a directory
#         y_path              is the path for the image masks
#         input_image_size    tuple of image size e.g. (250,250)
#         batch_size   size of the batches e.g. 16, 32 etc.
#         """
#         self.x, self.y = x_images, y_masks
#         self.x_path, self.y_path = x_path, y_path
#         self.input_image_size = input_image_size
#         self.batch_size = batch_size

#     def __len__(self):
#         """
#         return the number of batches required for the amount of
#         images in x_images
#         """
#         return math.ceil(len(self.x) / self.batch_size)

#     def __getitem__(self,batch_index_position):
#         # batch_index_position denotes start of current batch
#         # current_index determines end index of current batch
#         current_index = batch_index_position * self.batch_size

#         # create lists of the x and y image names in the current batch
#         batch_x = self.x[current_index:current_index+self.batch_size]
#         batch_y = self.y[current_index:current_index+self.batch_size]

#         # UNET_INPUT_SHAPE = (224,224,3)

#         self.resized_size = UNET_INPUT_SHAPE

#         # create a list for the tensorflow objects then read in images
#         # and convert to tensorflow objects.
#         xl = []
        
#         for i, ximg in enumerate(batch_x):
#             img = tf.io.read_file(self.x_path+batch_x[i])
#             img = tf.io.decode_jpeg(img, channels=3)
#             # resizing
#             input_img =  tf.image.resize(img, [self.resized_size[0], self.resized_size[1]])
#             # normalise
#             input_img = tf.math.divide(input_img, 255)
#             xl.append(input_img)
#         x = tf.stack(xl)

#         # create a list for the tensorflow objects then read in images
#         # and convert to tensorflow objects.
#         # mask images are single channel
#         yl = []
#         for j, yimg in enumerate(batch_y):
#             mask = tf.io.read_file(self.y_path+batch_y[i])
#             mask = tf.io.decode_jpeg(mask, channels=1)
#             # resizing
#             mask_img =  tf.image.resize(mask, [self.resized_size[0], self.resized_size[1]])
#             # normalise
#             mask_img = tf.math.divide(mask_img, 255)
#             yl.append(mask_img)
#         y = tf.stack(yl)

#         return x,y


In [14]:
# data.py

In [15]:
# import numpy as np
# import pandas as pd
# import os

# class GetData():
#     '''
#     Custom function to generate train, validation and optional test datasets
#     by specifying percentages. Saves to a csv file for reading in and out.
#     '''

#     def __init__(self,train_path,test_path,train_pc,val_pc,test_pc = 0.0):
#         '''
#         Specify train and ground truth ("test") paths
#         train_pc = percentage of images for training
#         val_pc   = percentage of images for validation purposes
#         test_pc  = [optional] reserve percentage of images for testing
#                    default is 0.0%
#         '''
#         self.train_path = train_path
#         self.test_path  = test_path
#         # Percentages
#         self.train_pc = 1.0 - (val_pc + test_pc)
#         self.val_pc = 1.0 - (train_pc + test_pc)
#         self.test_pc = test_pc

#     def make_dataframe(self):
#         '''
#         Create a dataframe of all available images in the train and test paths
#         '''
#         train_list = os.listdir(self.train_path)
#         train_list.sort()

#         test_list  = os.listdir(self.test_path)
#         test_list.sort()

#         self.data_df = pd.DataFrame(list(zip(train_list,test_list)), columns=['x_data','y_data'])
#         return self.data_df

#     def check_data(self,df,check_set):
#         '''
#         Check that the x and y data matches
#         '''
#         self.check = 0 # flag to allow later steps to proceed if datasets match

#         validate = (df.x_data == df.y_data.str.replace("_mask","")).sum()
#         if validate == len(df):
#             self.check = 1
#             return self.check
#         else:
#             return self.check

#     def data_split(self):
#         '''
#         Split the dataset according to the pre-defined percentages
#         Logic curated in specific way to ensure that valid int values are used
#         '''
#         n_train = int(len(self.data_df) * self.train_pc)
#         # to insure equal splits of integer values, validation is checked against
#         # the test set if it exists
#         if self.test_pc > 0.0:
#             n_val   = int(len(self.data_df) * self.val_pc)
#             n_test  = len(self.data_df) - n_train - n_val
#         # if no test set specified then n_val is just the total length minus train set
#         else:
#             n_val   = len(self.data_df) - n_train

#         # create a dataframe to pull out lists without replacement
#         data_split = self.data_df.copy()

#         # create the train df and then remove the train rows:
#         self.data_train_df = data_split.sample(n=n_train,random_state=42)
#         data_split.drop(data_split.index[[tuple(self.data_train_df.index)]])

#         # create the validation df and then remove the val rows
#         # still checks to see if a test set is being requested
#         if self.test_pc > 0.0:
#             self.data_val_df = data_split.sample(n=n_val,random_state=42)
#             data_split.drop(data_split.index[[tuple(self.data_val_df.index)]])
#             # test df takes what is remaining
#             self.data_test_df = data_split.copy()
#         else:
#             self.data_val_df = data_split.copy()

#     def get_datasets(self):
#         '''
#         Create the dataframes and convert to dictory with lists
#         additionally perform checks on the data before proceeding
#         '''
#         self.make_dataframe()
#         self.data_split()
#         self.data_dict = {'train_x':[],
#                           'train_y':[],
#                           'val_x':[],
#                           'val_y':[],
#                           'test_x':[],
#                           'test_y':[]}

#         compiled_checks = 0
#         ### CHECKS ###
#         if self.check_data(self.data_train_df,"Training Data") == 0:
#             return print("Training Data: ERROR ***UNMATCHED*** datasets, please go back and check input paths and image directories")
#         compiled_checks += 1

#         if self.check_data(self.data_val_df,  "Validation Data") == 0:
#             return print("Training Data: ERROR ***UNMATCHED*** datasets, please go back and check input paths and image directories")
#         compiled_checks += 1

#         if self.test_pc > 0.0:
#             if self.check_data(self.data_test_df, "Testing Data") == 0:
#                 return print("Test Data: ERROR ***UNMATCHED*** datasets, please go back and check input paths and image directories")
#             compiled_checks += 1

#         if self.val_pc > 0.0:
#             dict = self.make_dict(with_val = 1)
#         else:
#             dict = self.make_dict(with_val = 0)

#         return dict

#     def make_dict(self,with_val):
#         '''
#         make dictionary of the datasets
#         '''
#         if with_val == 1:
#             print("Datasets match, proceed")
#             self.data_dict['train_x'] = list(self.data_train_df.x_data)
#             self.data_dict['train_y'] = list(self.data_train_df.y_data)
#             self.data_dict['val_x'] = list(self.data_val_df.x_data)
#             self.data_dict['val_y'] = list(self.data_val_df.y_data)
#             self.data_dict['test_x'] = list(self.data_test_df.x_data)
#             self.data_dict['test_y'] = list(self.data_test_df.y_data)

#         elif with_val == 0:
#             print("Datasets match, proceed")
#             self.data_dict['train_x'] = list(self.data_train_df.x_data)
#             self.data_dict['train_y'] = list(self.data_train_df.y_data)
#             self.data_dict['val_x'] = list(self.data_val_df.x_data)
#             self.data_dict['val_y'] = list(self.data_val_df.y_data)

#         return self.data_dict

#     def save_datasets(self, save_path):
#         '''
#         allows for saving of dataframes so can consistently use the same datasets
#         Specifiy save path that is DIFFERENT from the images paths
#         '''
#         self.data_train_df.to_csv(save_path+"train_dataset.csv")
#         self.data_val_df.to_csv(save_path+"validation_dataset.csv")
#         if self.test_pc > 0.0:
#             self.data_test_df.to_csv(save_path+"test_dataset.csv")

# class LoadDataSets(GetData):
#     '''
#     Load the pre-saved lists of shuffled images back in and
#     produce the list dictionary
#     '''

#     def __init__(self, load_train, load_val, load_test = "none"):
#         '''
#         allows for loading of already generated datasets
#         input full filepath for each dataset to load
#         load_train  - Full filepath and filename of training .csv file
#         load_val    - Full filepath and filename of validation .csv file
#         load_test   - Full filepath and filename of test .csv file
#         '''
#         self.load_train = load_train
#         self.load_val = load_val
#         self.load_test = load_test

#     def load_datasets(self):
#         self.data_dict = {'train_x':[],
#                       'train_y':[],
#                       'val_x':[],
#                       'val_y':[],
#                       'test_x':[],
#                       'test_y':[]}
#         self.data_train_df = pd.read_csv(self.load_train, index_col = 0)
#         self.data_val_df   = pd.read_csv(self.load_val, index_col = 0)


#         if self.load_test != "none":
#             self.data_test_df = pd.read_csv(self.load_test, index_col = 0)
#             dict = self.make_dict(with_val = 1)
#         else:
#             dict = self.make_dict(with_val = 0)

#         return dict


# model.py

In [16]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, ReLU
from tensorflow.keras.layers import BatchNormalization, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model

from tensorflow.keras.applications import VGG16




class SMR_Model():
    ''' creating our first lwb_smr models '''

    def __init__(self, input_shape):
        self.input_shape = input_shape

    def get_latest_model(self):
        model = self.build_vgg16_unet(self.input_shape)
        model = self.compile_model(model)

        return model

    def convolution_block(self, inputs, num_filters):
        ''' simple UNET convolution block with BatchNormalisation '''

        # convolution layer 1 of the block
        x = Conv2D(num_filters, (3,3), padding='same')(inputs)  # padding='same' to avoid cut-down with conv
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        # convolution layer 2 of the block
        x = Conv2D(num_filters, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        # max pooling not used here as just the bridge

        return x

    def decoder_block(self, inputs, skip_tensor, num_filters):
        ''' decoder block for UNET '''
        # adds in the skips with concatenate
        x = Conv2DTranspose(num_filters, (2,2), strides=2, padding='same')(inputs) # stride important here to up-sample
        x = Concatenate()([x, skip_tensor])     # bringing in skip layer
        x = self.convolution_block(x, num_filters)

        return x

    def build_vgg16_unet(self, input_shape):
        ''' build vgg-16 '''

        inputs = Input(input_shape)

        # see actual VGG-16 here: https://github.com/keras-team/keras/blob/v2.9.0/keras/applications/vgg16.py#L43-L227
        vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=inputs)
        # vgg16.summary()
        vgg16.trainable = False

        ''' Encoder - skip layers '''
        skip1 = vgg16.get_layer('block1_conv2').output #  256 x 256, 64 filters in vgg16
        skip2 = vgg16.get_layer('block2_conv2').output #  128 x 128, 128 filters in vgg16
        skip3 = vgg16.get_layer('block3_conv3').output #   64 x 64, 256 filters in vgg16
        skip4 = vgg16.get_layer('block4_conv3').output #   32 x 32, 512 filters in vgg16
        # display('skip4: ' + str(skip4.shape))

        # only need to specify the skip layers, as VGG16 is an Encoder
        # Therefore, VGG16 comes built with MaxPool2d, so we don't specify

        ''' Bridge '''
        bridge = vgg16.get_layer('block5_conv3').output # 16 x 16, with 512 filters in vgg16
        # display('bridge: ' + str(bridge.shape))


        ''' Decoder '''
        d1 = self.decoder_block(bridge, skip4, 512) #  512 filters, as per the bridge
        d2 = self.decoder_block(d1, skip3, 256) #  256 filters
        d3 = self.decoder_block(d2, skip2, 128) #  128 filters
        d4 = self.decoder_block(d3, skip1, 64)  #   64 filters

        ''' Output '''
        outputs = Conv2D(1, (1,1), padding='same', activation='sigmoid')(d4)

        model = Model(inputs, outputs, name='first_VGG16_UNET')

        return model
    
    def dice_loss(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.math.sigmoid(y_pred)
        numerator = 2 * tf.reduce_sum(y_true * y_pred)
        denominator = tf.reduce_sum(y_true + y_pred)

        return 1 - numerator / denominator


    def compile_model(self, m):
        ''' with accuracy, binaryIoU, AuC '''
        # metrics
        threshold = 0.5
        binaryIoU = tf.keras.metrics.BinaryIoU(target_class_ids=[1], threshold=threshold)
        AuC = tf.keras.metrics.AUC()

        # loss
        #self.dice_loss = ...
        
        # Compile Model
        m.compile(
                    loss=self.dice_loss,
                    optimizer='adam',
                    metrics=['accuracy', binaryIoU, AuC]
                    )
        return m


# utils.py

In [17]:
# mlflow
#
# and others...

import mlflow
from mlflow.tracking import MlflowClient
from memoized_property import memoized_property

MLFLOW_URI = "https://mlflow.lewagon.ai/"

class PushMLFlow():
    '''
        MLFLOW_URI = "https://mlflow.lewagon.ai/"
        EXPERIMENT_NAME = "[UK] [LONDON] [SOLAR_ROOF] TEST RUN" # template
        EXPERIMENT_TAGS = {
            'USER': 'test_user',
            'RUN NAME': 'test runs',
            'VERSION': '1.0.1',
            'LOSS': 'dice'
            'DESCRIPTION': 'testing MLFlow Pipeline. Model - basic U-Net structure, 2 epochs, 15 images'
        }
    '''

    def __init__(self, experiment_name, experiment_tags):
        self.experiment_name = experiment_name
        self.experiment_tag = experiment_tags

    @memoized_property
    def mlflow_client(self):
        mlflow.set_tracking_uri(MLFLOW_URI)
        return MlflowClient()

    @memoized_property
    def mlflow_experiment_id(self):
        try:
            return self.mlflow_client.create_experiment(self.experiment_name)
        except BaseException:
            return self.mlflow_client.get_experiment_by_name(self.experiment_name).experiment_id

    @memoized_property
    def mlflow_run(self):
        return self.mlflow_client.create_run(self.mlflow_experiment_id, tags=self.experiment_tag)

    def mlflow_log_param(self, key, value):
        self.mlflow_client.log_param(self.mlflow_run.info.run_id, key, value)

    def mlflow_log_metric(self, key, value):
        self.mlflow_client.log_metric(self.mlflow_run.info.run_id, key, value)


# trainer.py

In [18]:

import os

import tensorflow as tf

# from lwb_smr.CustomDataLoader import CustomDataLoader
# from lwb_smr.model import SMR_Model
# from lwb_smr.utils import PushMLFlow

from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import AUC, IoU

        
def dice_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)

    return 1 - numerator / denominator

class Trainer():
    def __init__(self):
        pass

    def set_model(self, loss=dice_loss):
        self.loss = loss
        # Instantiate Model
        # our_input_shape = (224,224,3)

        getVGG16 = SMR_Model(UNET_INPUT_SHAPE)
        self.model = getVGG16.get_latest_model()
        # see compile in SMR_Model

    def start_mlflow(self):
        p = PushMLFlow(EXPERIMENT_NAME, EXPERIMENT_TAGS)
        return p # returns a class instance of PushMLFlow
    
    def run(self):

        print(80*'-')
        print('------SETTING FOR DATA RUN------')

        ### 
        ##
        #
        # customdata = self.just_get_the_data_loaded()

        print(80*'-')
        print('------MODEL RUNNING------')

        # set mflow      
        self.MFLOW = self.start_mlflow() # class instance of MLFLOW

        
        # set model
        self.set_model()
        
        TEST_CHECKPOINT_PATH_N_FILE_NAME = '../../../raw_data/checkpoints/test_220612_checkpoint_VGG16_Dice.h5'
        TEST_MODEL_PATH_N_FILE_NAME = '../../../raw_data/models/test_220612_UNET_VGG16_Dice_224x224x3.h5'

        mc = ModelCheckpoint(TEST_CHECKPOINT_PATH_N_FILE_NAME, save_best_only=True) # could put path here
        es = EarlyStopping(patience=15, restore_best_weights=True)
        self.history = self.model.fit(
            ds_train,
            validation_data=ds_val,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            callbacks=[mc, es]
            )

        # model_path_and_filename = 'test_220612_UNET_VGG16_Dice_224x224x3.h5'
        self.model.save(TEST_MODEL_PATH_N_FILE_NAME)

        self.MFLOW.mlflow_log_param('loss', self.loss)

        print(80*'=')
        print('------MODEL RUN SUCCESFULLY COMPLETED------')

        self.evaluate()

    def evaluate(self):
        print(80*'-')
        print('------MODEL EVALUATING------')
        results = self.model.evaluate(ds_val)
        for result in results:
            self.MFLOW.mlflow_log_metric('metric X val output', result)
        print(80*'=')
        print('------MODEL EVALUATED------')

if __name__ == '__main__':
    pass
    t = Trainer()
    t.run()
    t.history

--------------------------------------------------------------------------------
------SETTING FOR DATA RUN------
--------------------------------------------------------------------------------
------MODEL RUNNING------


2022-06-12 04:55:08.619697: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 576.0KiB (rounded to 589824)requested by op Mul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-06-12 04:55:08.619746: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-06-12 04:55:08.619760: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 15, Chunks in use: 15. 3.8KiB allocated for chunks. 3.8KiB in use in bin. 569B client-requested in use in bin.
2022-06-12 04:55:08.619767: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 1, Chunks in use: 1. 512B allocated for chunks. 512B in use in bin. 512B client-requested in use in bin.
2022-06-12 04:55:08.619774: I tensorflow/core/common

ResourceExhaustedError: failed to allocate memory [Op:Mul]

In [ ]:
########
####
###
#
#        NOTES:  
#                  
#
##
###
####
########